In [34]:
import glob
import os
import re
import pandas as pd

In [35]:
# helps sort file by number at the end of filename (10 correctly placed after 9 in filename)
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [42]:
ejmr_data = sorted(glob.glob('ejmr_pg*.csv'), key=numericalSort)


# pandas solution

df_list = []

for filename in sorted(ejmr_data):
    df_list.append(pd.read_csv(filename))
full_df = pd.concat(df_list)

full_df.to_csv('ejmr_data.csv', index=True, index_label='pandas_index')
